In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import json
import urllib
# from urlparse import urlparse
from urllib.parse import urlparse
import httplib2 as http #External library
import pandas as pd
from tqdm import tqdm
from onemapsg import OneMapClient
import time
import networkx as nx
import geopy.distance

In [71]:
#Authentication parameters
headers = { 'AccountKey' : 'esuwl3rGSEqFBc+rSpuWEg==',
            'accept' : 'application/json'} #this is by default

uri = 'http://datamall2.mytransport.sg' #Resource URL

bus_route_path = '/ltaodataservice/BusRoutes?$skip={}'
bus_route_out_file = "bus_routes{}.json"

bus_stops_path = '/ltaodataservice/BusStops?$skip={}'
bus_stop_out_file = "bus_stops{}.json"

bus_service_path = '/ltaodataservice/BusServices?$skip={}'
bus_service_out_file = "bus_services{}.json"

In [4]:
def extract_data_datamall(headers,uri,path,save_path):
    data_list = []
    lim = 0
    control_var = True
    while(control_var):
#     for lim in range(0, 25501, 500):
        #API parameters
        url_path = path.format(lim)
        #Build query string & specify type of API call
        target = urlparse(uri + url_path)
        method = 'GET'
        body = ''
        #Get handle to http
        h = http.Http()
        #Obtain results
        response, content = h.request(target.geturl(),method,body,headers)
        #Parse JSON to print
        jsonObj = json.loads(content)
        if(len(jsonObj['value'])==0):
            control_var = False
        data_list.extend(jsonObj['value'])
        lim = lim + 500

    return data_list
  

In [28]:
def read_json(file_path):
    f = open(file_path)
    data = json.load(f)
    f.close()
    
    return data

## Bus stop info

In [72]:
bus_routes_all = extract_data_datamall(headers,uri,bus_route_path,bus_route_out_file)
print("exracted all bus routes")
bus_stop_info = extract_data_datamall(headers,uri,bus_stops_path,bus_stop_out_file)
print("extrcated all bus stops")
bus_services_all = extract_data_datamall(headers,uri,bus_service_path,bus_service_out_file)
print("exracted all bus services")

exracted all bus routes
extrcated all bus stops
exracted all bus services


In [73]:
print(len(bus_routes_all))
print(len(bus_stop_info))

25146
5083


In [74]:
services = {}
for s in bus_services_all:
    service_no = s['ServiceNo']
    services[service_no] = {}
    for freq in ['AM_Peak_Freq', 'AM_Offpeak_Freq', 'PM_Peak_Freq', 'PM_Offpeak_Freq']:
        try:
            mean_list = [int(i) for i in s[freq].split('-')]
            services[service_no][freq] = sum(mean_list) / len(mean_list)
        except:
            services[service_no][freq] = None

In [77]:
bus_stop_info_processed = {}

for deets in bus_stop_info:
    bus_stop_info_processed[deets['BusStopCode']] = deets
    bus_stop_info_processed[deets['BusStopCode']].pop('BusStopCode')

In [78]:
# bus_stop_volume_aug = pd.read_csv('/kaggle/input/input-data/transport_node_bus_202208.csv')
bus_stop_volume_aug = pd.read_csv('./transport_node_bus_202208.csv')
bus_stop_volume_aug['PT_CODE'] = bus_stop_volume_aug['PT_CODE'].astype(str)
bus_stop_volume_aug['PT_CODE'] = bus_stop_volume_aug['PT_CODE'].apply(lambda x: '0'+x if len(x) < 5 else x)
bus_stop_volume_aug.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2022-08,WEEKDAY,6,BUS,67551,2878,100
1,2022-08,WEEKENDS/HOLIDAY,6,BUS,67551,209,14
2,2022-08,WEEKENDS/HOLIDAY,7,BUS,66541,141,119
3,2022-08,WEEKDAY,7,BUS,66541,608,287
4,2022-08,WEEKENDS/HOLIDAY,17,BUS,54209,535,164


In [79]:
bus_stop_volume_aug['TOTAL_TAP_IN_VOLUME'] = bus_stop_volume_aug['TOTAL_TAP_IN_VOLUME'].astype(int)
bus_stop_volume_aug['TOTAL_TAP_OUT_VOLUME'] = bus_stop_volume_aug['TOTAL_TAP_OUT_VOLUME'].astype(int)
bus_stop_volume_aug.dtypes

YEAR_MONTH              object
DAY_TYPE                object
TIME_PER_HOUR            int64
PT_TYPE                 object
PT_CODE                 object
TOTAL_TAP_IN_VOLUME      int64
TOTAL_TAP_OUT_VOLUME     int64
dtype: object

In [80]:
bus_stop_vol_aug = bus_stop_info_processed.copy()

for i in tqdm(range(len(bus_stop_volume_aug))):
    value = bus_stop_vol_aug.get(bus_stop_volume_aug.iloc[i]['PT_CODE'],False)

    if(value == False):
        bus_stop_vol_aug[bus_stop_volume_aug.iloc[i]['PT_CODE']] = {'weekdays':{'tap_in':[0]*24,'tap_out':[0]*24},'weekends':{'tap_in':[0]*24,'tap_out':[0]*24}}
    dict_keys = bus_stop_vol_aug[bus_stop_volume_aug.iloc[i]['PT_CODE']].keys()
    if('weekdays' not in dict_keys or 'weekends' not in dict_keys):
        bus_stop_vol_aug[bus_stop_volume_aug.iloc[i]['PT_CODE']].update({'weekdays':{'tap_in':[0]*24,'tap_out':[0]*24},'weekends':{'tap_in':[0]*24,'tap_out':[0]*24}})

    if(bus_stop_volume_aug.iloc[i]['DAY_TYPE'] == 'WEEKDAY'):
        bus_stop_vol_aug[bus_stop_volume_aug.iloc[i]['PT_CODE']]['weekdays']['tap_in'][bus_stop_volume_aug.iloc[i]['TIME_PER_HOUR']] = int(bus_stop_volume_aug.iloc[i]['TOTAL_TAP_IN_VOLUME'])
        bus_stop_vol_aug[bus_stop_volume_aug.iloc[i]['PT_CODE']]['weekdays']['tap_out'][bus_stop_volume_aug.iloc[i]['TIME_PER_HOUR']] = int(bus_stop_volume_aug.iloc[i]['TOTAL_TAP_OUT_VOLUME'])
    else:
        bus_stop_vol_aug[bus_stop_volume_aug.iloc[i]['PT_CODE']]['weekends']['tap_in'][bus_stop_volume_aug.iloc[i]['TIME_PER_HOUR']] = int(bus_stop_volume_aug.iloc[i]['TOTAL_TAP_IN_VOLUME'])
        bus_stop_vol_aug[bus_stop_volume_aug.iloc[i]['PT_CODE']]['weekends']['tap_out'][bus_stop_volume_aug.iloc[i]['TIME_PER_HOUR']] = int(bus_stop_volume_aug.iloc[i]['TOTAL_TAP_OUT_VOLUME'])

100%|██████████| 195803/195803 [03:08<00:00, 1040.64it/s]


In [81]:
service_no_dict = {}

for deets in bus_routes_all:
    value = service_no_dict.get(deets['ServiceNo'],False)

    if(value == False):
        service_no_dict[deets['ServiceNo']] = {'busstop':[],'initial_dist':[],'busstop_info':{}}

    service_no_dict[deets['ServiceNo']]['busstop'].append(deets['BusStopCode'])
    service_no_dict[deets['ServiceNo']]['initial_dist'].append(deets['Distance'])

    busstop_info = bus_stop_info_processed.get(deets['BusStopCode'],{})
    service_no_dict[deets['ServiceNo']]['busstop_info'][deets['BusStopCode']] = busstop_info
    service_no_dict[deets['ServiceNo']]['busstop_info'][deets['BusStopCode']].update(bus_stop_vol_aug[deets['BusStopCode']])

    
    service_no_dict[deets['ServiceNo']].update(services[deets['ServiceNo']])

In [82]:
bus_stop_full_info = {}
for keys in service_no_dict.keys():
    bus_stop_full_info.update(service_no_dict[keys]['busstop_info'])

In [99]:
def write_to_json(dict_obj,file_name):
    with open(file_name, "w") as outfile:
        json.dump(dict_obj, outfile)
    print('written file to '+file_name)

In [84]:
write_to_json(service_no_dict,'new_service_no_dict.json')

written file to new_service_no_dict.json


## Train stations info

In [85]:
# train_station = pd.read_csv('/kaggle/input/input-data/Train Station Codes and Chinese Names.csv')
train_station = pd.read_csv('./Train Station Codes and Chinese Names.csv')
train_station.head()

,stn_code,mrt_station_english,mrt_station_chinese,mrt_line_english,mrt_line_chinese
0,NS1,Jurong East,裕廊东,North-South Line,南北线
1,NS2,Bukit Batok,武吉巴督,North-South Line,南北线
2,NS3,Bukit Gombak,武吉甘柏,North-South Line,南北线
3,NS4,Choa Chu Kang,蔡厝港,North-South Line,南北线
4,NS5,Yew Tee,油池,North-South Line,南北线


In [86]:
train_station_names = {}

for i in range(len(train_station)):
    try:
        train_station_names[train_station.iloc[i]['mrt_station_english']] = train_station_names[train_station.iloc[i]['mrt_station_english']] + '-' + train_station.iloc[i]['stn_code']
    except:
        train_station_names[train_station.iloc[i]['mrt_station_english']] = train_station.iloc[i]['stn_code']

In [87]:
new_stn_code = []
for i in range(len(train_station)):
    new_stn_code.append(train_station_names[train_station.iloc[i]['mrt_station_english']])

train_station['new_stn_code'] = new_stn_code

In [88]:
train_station_dict = {}

for i in tqdm(range(len(train_station))):
    value = train_station_dict.get(train_station.iloc[i]['mrt_line_english'],False)
    if(value == False):
        train_station_dict[train_station.iloc[i]['mrt_line_english']] = {'stn_code':[],'stn_info':{}}
        
    train_station_dict[train_station.iloc[i]['mrt_line_english']]['stn_code'].append(train_station.iloc[i]['new_stn_code'])
    # train_station_dict[train_station.iloc[i]['mrt_line_english']]['stn_info']['stn_name'] = train_station.iloc[i]['mrt_station_english']
    
    Client = OneMapClient("vaidyanathanabhishek@gmail.com", "ShakeShake033@")
    data_dict = Client.search(train_station.iloc[i]['stn_code'])['results'][0]

    # time.sleep(5)
  
    train_station_dict[train_station.iloc[i]['mrt_line_english']]['stn_info'][train_station.iloc[i]['new_stn_code']] = {'stn_name':train_station.iloc[i]['mrt_station_english']}
    train_station_dict[train_station.iloc[i]['mrt_line_english']]['stn_info'][train_station.iloc[i]['new_stn_code']].update(data_dict)

100%|██████████| 193/193 [00:20<00:00,  9.33it/s]


In [89]:
train_station_full_info = {}

for keys in train_station_dict:
    train_station_full_info.update(train_station_dict[keys]['stn_info'])

## Save to json files

In [90]:
write_to_json(service_no_dict,'service_no_dict.json')
write_to_json(bus_stop_full_info,'bus_stop_full_info.json')
write_to_json(train_station_dict,'train_station_dict.json')
write_to_json(train_station_full_info,'train_station_full_info.json')
write_to_json(bus_stop_info_processed,'bus_stop_info_processed.json')

written file to service_no_dict.json
written file to bus_stop_full_info.json
written file to train_station_dict.json
written file to train_station_full_info.json
written file to bus_stop_info_processed.json


## NetworkX - Graph

In [91]:
# coords_1 = (bus_stop_full_info['28201']['Latitude'], bus_stop_full_info['28201']['Longitude'])
# coords_2 = (train_station_full_info['NS1-EW24']['LATITUDE'], train_station_full_info['NS1-EW24']['LONGITUDE'])

# print (geopy.distance.geodesic(coords_1, coords_2))

0.1305111865106769 km


In [29]:
service_no_dict = read_json("./new_service_no_dict.json")

In [92]:
# bus_stop_nodes = list(bus_stop_vol_aug.keys())

In [30]:
# transport_graph = nx.DiGraph()
# for key in service_no_dict.keys():
#     i = 0
#     while i < (len(service_no_dict[key]['busstop'])-1):
#         first_node = service_no_dict[key]['busstop'][i]
#         second_node = service_no_dict[key]['busstop'][i+1]
#         lat1 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i]]['Latitude']
#         long1 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i]]['Longitude']
#         lat2 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i+1]]['Latitude']
#         long2 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i+1]]['Longitude']
#         transport_graph.add_node(first_node, type = 'bus_stop',latitude = float(lat1), longitude=float(long1))
#         transport_graph.add_node(second_node, type = 'bus_stop',latitude = float(long1), longitude = float(long2))
#         transport_graph.add_edge(first_node,second_node, type =  'bus_bus',distance=3)

#         i = i+1

# # edge_labels = dict([((n1, n2), {'labels':'bus_route'})
# #                     for n1, n2 in bus_routes_graph.edges])
# # nx.set_edge_attributes(bus_routes_graph, edge_labels)
bus_transport_graph = nx.DiGraph()
for key in service_no_dict.keys():
    i = 0
    while i < (len(service_no_dict[key]['busstop'])-1):
        first_node = service_no_dict[key]['busstop'][i]
        second_node = service_no_dict[key]['busstop'][i+1]
        busstop_dist = service_no_dict[key]['initial_dist'][i+1] - service_no_dict[key]['initial_dist'][i]
        # if(busstop_dist<0 and first_node!=second_node):
        #     print(first_node,second_node)
        #     print("hello:",(service_no_dict[key]['initial_dist'][i+1],service_no_dict[key]['initial_dist'][i]))
        lat1 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i]]['Latitude']
        long1 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i]]['Longitude']
        lat2 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i+1]]['Latitude']
        long2 = service_no_dict[key]['busstop_info'][service_no_dict[key]['busstop'][i+1]]['Longitude']
        bus_transport_graph.add_node(first_node, type = 'bus_stop',latitude = float(lat1), longitude=float(long1))
        bus_transport_graph.add_node(second_node, type = 'bus_stop',latitude = float(lat2), longitude = float(long2))
        bus_transport_graph.add_edge(first_node,second_node, type =  'bus_bus',weight=abs(busstop_dist))

        i = i+1

# edge_labels = dict([((n1, n2), {'labels':'bus_route'})
#                     for n1, n2 in bus_routes_graph.edges])
# nx.set_edge_attributes(bus_routes_graph, edge_labels)

In [94]:
train_transport_graph = nx.Graph()
for key in train_station_dict.keys():
    i = 0
    while i < (len(train_station_dict[key]['stn_code'])-1):
        first_node = train_station_dict[key]['stn_code'][i]
        second_node = train_station_dict[key]['stn_code'][i+1]
        lat1 = train_station_dict[key]['stn_info'][train_station_dict[key]['stn_code'][i]]['LATITUDE']
        long1 = train_station_dict[key]['stn_info'][train_station_dict[key]['stn_code'][i]]['LONGITUDE']
        lat2 = train_station_dict[key]['stn_info'][train_station_dict[key]['stn_code'][i+1]]['LATITUDE']
        long2 = train_station_dict[key]['stn_info'][train_station_dict[key]['stn_code'][i+1]]['LONGITUDE']
        train_transport_graph.add_node(first_node, type = 'mrt_station', latitude = float(lat1), longitude = float(long1))                                             
        train_transport_graph.add_node(second_node, type = 'mrt_station',latitude = float(lat2), longitude = float(long2))
        train_transport_graph.add_edge(first_node,second_node,type = 'mrt_mrt')
        # train_transport_graph.add_edge(second_node,first_node,type = 'mrt_mrt')

        i = i+1

# edge_labels = dict([((n1, n2), {'labels':'train_route'})
#                     for n1, n2 in train_routes_graph.edges])
# nx.set_edge_attributes(train_routes_graph, edge_labels)

In [95]:
bus_stop_nodes = {}
mrt_station_nodes = {}
for (p, d) in bus_transport_graph.nodes(data=True):
    if d['type'] == 'bus_stop':
        bus_stop_nodes[p] = d
    if(d['type'] == 'mrt_station'):
        mrt_station_nodes[p] = d

bus_stop_nodes = {}
mrt_station_nodes = {}
for (p, d) in train_transport_graph.nodes(data=True):
    if d['type'] == 'bus_stop':
        bus_stop_nodes[p] = d
    if(d['type'] == 'mrt_station'):
        mrt_station_nodes[p] = d

In [96]:
# for mrt_keys in tqdm(mrt_station_nodes.keys()):
#     for bus_keys in bus_stop_nodes.keys():
#         coords_1 = (mrt_station_nodes[mrt_keys]['latitude'], mrt_station_nodes[mrt_keys]['longitude'])
#         coords_2 = (bus_stop_nodes[bus_keys]['latitude'], bus_stop_nodes[bus_keys]['longitude'])
#         try:
#             distance = geopy.distance.geodesic(coords_1, coords_2)
#             if(distance < 0.5):
#                 transport_graph.add_edge(mrt_keys,bus_keys,type = 'bus_mrt')
#                 transport_graph.add_edge(bus_keys,mrt_keys,type = 'bus_mrt')
#         except:
# #             print(coords_1)
# #             print(coords_2)
#             pass

In [97]:
# red = ((u,v) for u,v,d in transport_graph.edges(data=True) if d['type']=='bus_mrt')
# data1 = nx.node_link_data(G)
# s1 = json.dumps(data1)

In [32]:
from networkx.readwrite import json_graph
    
def save_graph_data(file_name,graph):
    data = json_graph.node_link_data(graph)
    with open(file_name, "w") as outfile:
        json.dump(data, outfile)
    print('written file to '+file_name)
    
save_graph_data('new_bus_transport_graph.json',bus_transport_graph)

written file to new_bus_transport_graph.json


# Load graph 

In [4]:
def read_json(file_path):
    f = open(file_path)
    data = json.load(f)
    f.close()
    
    return data

In [5]:
def load(fname):
    G = nx.DiGraph()
    # d = json.load(open(fname))
    d = read_json(fname)
    nodes_list = []
    bus_bus_links = []
    for nodes in d['nodes']:
        G.add_node(nodes['id'], type = nodes['type'], latitude = float(nodes['latitude']), longitude = float(nodes['longitude']))                                             
    for links in d['links']:
        G.add_edge(links['source'],links['target'],type = links['type'],weight=links['weight'])
    # G.add_nodes_from(nodes_list)
    # G.add_edges_from(bus_mrt_links,type="bus_mrt")
    # G.add_edges_from(bus_bus_links,type="bus_bus")
    # G.add_edges_from(mrt_mrt_links,type="mrt_mrt")
    return G

transport_graph = load('./bus_transport_graph.json')

In [10]:
passenger_volume_bus = pd.read_csv('./origin_destination_bus_202208.csv')
passenger_volume_bus.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2022-08,WEEKENDS/HOLIDAY,17,BUS,24499,22221,2
1,2022-08,WEEKENDS/HOLIDAY,10,BUS,65239,65159,2
2,2022-08,WEEKDAY,10,BUS,65239,65159,2
3,2022-08,WEEKDAY,7,BUS,23519,23311,1
4,2022-08,WEEKDAY,16,BUS,54349,53241,1


In [11]:
new_pass_vol = passenger_volume_bus.groupby(['ORIGIN_PT_CODE','DESTINATION_PT_CODE','DAY_TYPE'])['TOTAL_TRIPS'].sum().reset_index()
new_pass_vol.head()

,ORIGIN_PT_CODE,DESTINATION_PT_CODE,DAY_TYPE,TOTAL_TRIPS
0,1012,1112,WEEKDAY,1608
1,1012,1112,WEEKENDS/HOLIDAY,557
2,1012,1113,WEEKDAY,1083
3,1012,1113,WEEKENDS/HOLIDAY,468
4,1012,1121,WEEKDAY,742


In [15]:
%%time
def weight_fn(source,target,edge):
    if(edge['type'] == 'bus_bus'):
        return 1
    else:
        return 0

edge_dict = {}
# for i in tqdm(range(len(passenger_volume_bus))):
# for i in tqdm(range(len(new_pass_vol))):
for i in tqdm(range(100)):
    source = str(passenger_volume_bus.iloc[i]["ORIGIN_PT_CODE"])
    target = str(passenger_volume_bus.iloc[i]["DESTINATION_PT_CODE"])
    if(len(source)==4):
        source = '0'+source
    if(len(target)==4):
        target = '0'+target
    try:
        shortest_path = (nx.shortest_path(transport_graph, source=source, target=target,weight='weight'))
        for j in range(len(shortest_path)-1):
            val = edge_dict.get((shortest_path[j],shortest_path[j+1]),None)
            if(val is None):
                edge_dict[(shortest_path[j],shortest_path[j+1])] = {'weekends_trips':0,'weekdays_trips':0}
            if(passenger_volume_bus.iloc[i]['DAY_TYPE'] == 'WEEKENDS/HOLIDAY'):
                edge_dict[(shortest_path[j],shortest_path[j+1])]['weekends_trips'] += int(passenger_volume_bus.iloc[i]['TOTAL_TRIPS'])
            else:
                edge_dict[(shortest_path[j],shortest_path[j+1])]['weekdays_trips'] += int(passenger_volume_bus.iloc[i]['TOTAL_TRIPS'])
    except:
        pass

100%|██████████| 100/100 [00:00<00:00, 219.50it/s]

CPU times: user 419 ms, sys: 12 ms, total: 431 ms
Wall time: 489 ms


In [18]:
def remap_keys(mapping):
    return {str(k): v for k, v in mapping.items()}
write_to_json(remap_keys(edge_dict),'bus_route_freq_dict.json')

written file to bus_route_freq_dict.json


In [20]:
nx.set_edge_attributes(transport_graph, edge_dict)

# NetworkX to ne04j

In [3]:
# user = 'neo4j'
# password = 'C3DKuamyGb64q9f8ldtq4u0N9Jjxag-9eRPceC0MwPQ' #YOUR OWN SANDBOX PASSWORD
# uri = 'neo4j+s://2c5e8457.databases.neo4j.io' #YOUR OWN SANDBOX URL

In [4]:
# from neo4j import GraphDatabase
# import nxneo4j as nx

In [8]:
# # driver = GraphDatabase.driver(uri=uri,auth=(user,password))
# #                               #OR "bolt://localhost:7673" for Neo4j Desktop
# #                               #OR the cloud url
# uri      = "neo4j+s://2c5e8457.databases.neo4j.io:7687" # in Neo4j Desktop
# uri      = "neo4j+s://2c5e8457.databases.neo4j.io"
# # uri      = "bolt://localhost:7687"
#                               # custom URL for Sandbox or Aura
# user     = "neo4j"            # your user name 
#                               # default is always "neo4j" 
#                               # unless you have changed it. 
# password = 'C3DKuamyGb64q9f8ldtq4u0N9Jjxag-9eRPceC0MwPQ'
# driver = GraphDatabase.driver(uri=uri,auth=(user,password))

In [12]:
# transport_graph = nx.DiGraph(driver) # directed graph
# transport_graph.delete_all()  #This will delete all the data, be careful
#                 #Just making sure that the results are reprodusible.

In [16]:
# def read_json(file_path):
#     f = open(file_path)
#     data = json.load(f)
#     f.close()
    
#     return data

# def load(fname):
#     # G = nx.DiGraph()
#     G = nx.DiGraph(driver)
#     # d = json.load(open(fname))
#     d = read_json(fname)
#     nodes_list = []
#     bus_bus_links = []
#     for nodes in d['nodes'][:50]:
#         # G.add_node(nodes['id'], type = nodes['type'], latitude = float(nodes['latitude']), longitude = float(nodes['longitude']))
#         G.add_node(nodes['id'])                                               
#     for links in d['links'][:50]:
#         G.add_edge(links['source'],links['target'],type = links['type'],weight=links['weight'])
#     # G.add_nodes_from(nodes_list)
#     # G.add_edges_from(bus_mrt_links,type="bus_mrt")
#     # G.add_edges_from(bus_bus_links,type="bus_bus")
#     # G.add_edges_from(mrt_mrt_links,type="mrt_mrt")
#     return G

In [1]:
# G = load('./bus_transport_graph.json')
# nxneo.draw(G)

# Additional Processing

In [33]:
pop_estimate = pd.read_csv("./population_estimates.csv")
pop_estimate.head()

,id,lat,lng,planningArea,tap_in,tap_out,deficit,avg_flow,pop_multiplier,pop_estimate
0,75009,1.354076,103.943391,TAMPINES,4607736,4502560,105176,4555148,0.190125,49413
1,76059,1.352962,103.941652,TAMPINES,330559,116454,214105,223506,0.009329,2424
2,76069,1.348753,103.942086,TAMPINES,310013,204128,105885,257070,0.010730,2788
3,96289,1.340055,103.948381,TAMPINES,144639,312525,-167886,228582,0.009541,2479
4,96109,1.337371,103.950673,TAMPINES,74787,104606,-29819,89696,0.003744,973


In [63]:
bus_odn = pd.read_csv("./bus_odn.csv")
bus_odn.head()

,orig,dest,freq_type,avg_trips
0,1012,1112,AM_Offpeak_Freq,760.428571
1,1012,1112,AM_Peak_Freq,126.142857
2,1012,1112,PM_Offpeak_Freq,175.285714
3,1012,1112,PM_Peak_Freq,245.857143
4,1012,1113,AM_Offpeak_Freq,438.142857


In [67]:
add_cols = list(pop_estimate.columns)[3:]
add_attr = {}
for i in range(len(pop_estimate)):
    if(len(str(pop_estimate.iloc[i]['id']))<5):
        id_key = '0'+str(pop_estimate.iloc[i]['id'])
    else:
        id_key = str(pop_estimate.iloc[i]['id'])
    add_attr[id_key]  = {}
    for j in add_cols:
        add_attr[id_key][j] = pop_estimate.iloc[i][j]

In [59]:
bus_transport_graph_copy = bus_transport_graph.copy()
nx.set_node_attributes(bus_transport_graph_copy, add_attr)

In [73]:
add_cols = list(bus_odn.columns)[2:]
add_edge_attr = {}
# for i in tqdm(range(len(bus_odn))):
for i in tqdm(range(1000)):
    if(len(str(bus_odn.iloc[i]['orig']))<5):
        id1 = '0'+(str(bus_odn.iloc[i]['orig']))
    else:
        id1 = (str(bus_odn.iloc[i]['orig']))

    if(len(str(bus_odn.iloc[i]['dest']))<5):
        id2 = '0'+(str(bus_odn.iloc[i]['dest']))
    else:
        id2 = (str(bus_odn.iloc[i]['dest']))
    val = add_edge_attr.get((id1,id2),'not present')
    if(val == 'not present'):
        add_edge_attr[(id1,id2)] = {}
    
    add_edge_attr[(id1,id2)][bus_odn.iloc[i]['freq_type']] = bus_odn.iloc[i]['avg_trips']    

100%|██████████| 1000/1000 [00:00<00:00, 1362.61it/s]


In [75]:
nx.set_edge_attributes(bus_transport_graph_copy, add_edge_attr)

In [81]:
nx.get_edge_attributes(bus_transport_graph_copy,'AM_Offpeak_Freq')

{('01012', '01112'): 760.4285714285714,
 ('01012', '01113'): 438.1428571428571,
 ('01013', '01113'): 88.85714285714286,
 ('01013', '01112'): 111.42857142857144}

# Clean data

In [87]:
bus_transport_graph_extra_feat = read_json("./bus_transport_graph_extra_feat.json")

In [91]:
links_feat_keys = []
for i in bus_transport_graph_extra_feat['links']:
    links_feat_keys.extend(list(i.keys()))
    links_feat_keys = list(set(links_feat_keys))

nodes_feat_keys = []
for i in bus_transport_graph_extra_feat['nodes']:
    nodes_feat_keys.extend(list(i.keys()))
    nodes_feat_keys = list(set(nodes_feat_keys))

In [ ]:
'78031'

In [94]:
for i in range(len(bus_transport_graph_extra_feat['nodes'])):
    for j in nodes_feat_keys:
        if j not in bus_transport_graph_extra_feat['nodes'][i]:
            bus_transport_graph_extra_feat['nodes'][i][j] = 0

In [97]:
for i in range(len(bus_transport_graph_extra_feat['links'])):
    for j in links_feat_keys:
        if j not in bus_transport_graph_extra_feat['links'][i]:
            bus_transport_graph_extra_feat['links'][i][j] = 0

In [100]:
write_to_json(bus_transport_graph_extra_feat,'new_bus_transport_graph_extra_feat.json')

written file to new_bus_transport_graph_extra_feat.json
